In [17]:
import pickle
from datetime import datetime

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from tqdm import tqdm

from data_generation_interpolated_lib import InterpolatedDataGenerator

In [18]:
# interp_path = '/Users/oskar/MSci/new-physics/toy_data/data_generation/data/interpolation/interp_2022_1_31_0'
# interp_path = '/Users/oskar/MSci/new-physics/toy_data/data_generation/data/interpolation/interp_1.1-8_-3-3_2022_2_15_19'
interp_path = '/Users/oskar/MSci/new-physics/toy_data/data_generation/data/interpolation/interp_0.1-1_-3-3_2022_2_16_16'

In [29]:
# xx, yy = np.meshgrid(np.linspace(-3,3,100), np.linspace(-3,3,100))
xx, yy = np.meshgrid(np.arange(-3,4), np.arange(-3,4))
wilson_coefficients_grid = np.vstack([xx.flatten(), yy.flatten()]).T

In [33]:
data_generator = InterpolatedDataGenerator(interp_path)
data_generator.q2_max = 0.98
data_generator.q2_min = 0.1

c_min, c_max = -3, 3
n_bins = 10

number_points = 50_000
sample_points = 10_000

sample_points_grid = 5_000

wilson_coefficients = np.random.uniform(c_min, c_max, (number_points, 2))

bin_edges_qkl = [
    np.linspace(0.1, 0.98, n_bins+1),           # q2
    np.linspace(0, np.pi, n_bins+1),         # k
    np.linspace(0, np.pi, n_bins+1),         # l
    # np.linspace(-np.pi, np.pi, n_bins+1),         # p
]

# bin_edges_qklp = [
#     np.linspace(1.1, 8, n_bins+1),           # q2
#     np.linspace(0, np.pi, n_bins+1),         # k
#     np.linspace(0, np.pi, n_bins+1),         # l
#     np.linspace(-np.pi, np.pi, n_bins+1),         # p
# ]

In [28]:
dataset = []
for c9, c10 in tqdm(wilson_coefficients):
    sample = data_generator.generate_data(c9, c10, sample_points * 10)

    values_3d_qkl, _ = np.histogramdd(
        sample[['q2', 'k', 'l']].values,
        bins=bin_edges_qkl,
        # weights=sample['weight'],
        density=True
    )

    # values_3d_qklp, _ = np.histogramdd(
    #     sample[['q2', 'k', 'l', 'p']].values,
    #     bins=bin_edges_qklp,
    #     density=True
    # )

    dataset.append({
        'histogram_qkl': values_3d_qkl,
        # 'histogram_qklp': values_3d_qklp,
        'c9': c9,
        'c10': c10
    })


now = datetime.now()
output_file_name = f'dataset_q0.1-0.98_qkl_{n_bins}bins_{sample_points}n_{now.month}_{now.day}_{now.hour}.pkl'
path_to_file = 'data/'
with open(path_to_file + output_file_name, 'wb') as out_file:
    pickle.dump(dataset, out_file)


  0%|          | 1/50000 [00:00<5:44:41,  2.42it/s]


ValueError: One of the requested xi is out of bounds in dimension 2

In [34]:
dataset = []
for c9, c10 in tqdm(wilson_coefficients_grid):

    sample_big = data_generator.generate_data(c9, c10, 1_000_000)

    for _ in range(1000):
        sample = sample_big.sample(sample_points_grid)
        values_3d_qkl, _ = np.histogramdd(
            sample[['q2', 'k', 'l']].values,
            bins=bin_edges_qkl,
            # weights=sample['weight'],
            density=True
        )

        # values_3d_qklp, _ = np.histogramdd(
        #     sample[['q2', 'k', 'l', 'p']].values,
        #     bins=bin_edges_qklp,
        #     density=True
        # )

        dataset.append({
            'histogram_qkl': values_3d_qkl,
            # 'histogram_qklp': values_3d_qklp,
            'c9': c9,
            'c10': c10
        })


now = datetime.now()
output_file_name = f'dataset_q0.1-0.98_qkl_{n_bins}bins_{sample_points_grid}n_{now.month}_{now.day}_{now.hour}_grid.pkl'
path_to_file = 'data/'
with open(path_to_file + output_file_name, 'wb') as out_file:
    pickle.dump(dataset, out_file)


100%|██████████| 49/49 [08:35<00:00, 10.53s/it]
